In [1]:
import pandas as pd
import numpy as np
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH


# Cargar set de datos

In [2]:
trip = pd.read_csv('data/trip_train.csv')

In [3]:
trip_test = pd.read_csv('data/trip_test.csv')

In [4]:
weather = pd.read_csv('data/weather.csv')

In [5]:
station = pd.read_csv('data/station.csv')

# Modificando set de datos

In [6]:
trip_test['start_station_name'] = trip_test['start_station_name']\
    .apply(lambda x : 'Washington at Kearney' if x == 'Washington at Kearny' else x  )

In [7]:
trip_test['start_station_name'] = trip_test['start_station_name']\
    .apply(lambda x : 'Post at Kearney' if x == 'Post at Kearny' else x  )

In [8]:
otherStation = pd.DataFrame([['Broadway at Main',94107],['San Jose Government Center',95113]]\
                            ,columns=['station','zip_code'])

In [9]:
stationReduced = station.loc[:,['id','name','city']]

def choseZipCode(city):
    if city == 'San Francisco' : return 94107
    elif city == 'San Jose' : return 95113
    elif city == 'Redwood City' : return 94063
    elif city == 'Palo Alto' : return 94301
    elif city == 'Mountain View' : return 94041

stationReduced['zip_code'] = stationReduced.loc[:,'city'].apply(choseZipCode)

stationReduced = stationReduced.drop(labels=['id','city'],axis=1)
stationReduced.rename(columns={'name':'station'},inplace=True)

In [10]:
stationReduced = stationReduced.append(otherStation,ignore_index=True)

In [11]:
trip.rename(columns={'start_station_name':'station'},inplace=True)
trip_test.rename(columns={'start_station_name':'station'},inplace=True)

In [12]:
trip_test = trip_test.drop(labels=['zip_code','end_date','end_station_name','end_station_id','bike_id'],axis=1)

In [13]:
trip = trip.drop(labels=['zip_code','end_date','end_station_name','end_station_id','bike_id'],axis=1)

In [14]:
weather['precipitation_inches'] = weather['precipitation_inches'].apply(lambda x : 0.005 if (x == 'T') else float(x))

In [15]:

weather = weather[weather.precipitation_inches.isnull() == False]

weather = weather.fillna(0)

weather['events'] = weather['events'].apply(lambda x : 'Normal' if x == 0 else x)
weather['events'] = weather['events'].apply(lambda x : 'Rain' if x == 'rain' else x)
weather['date'] = weather['date'].apply(lambda date : pd.to_datetime(date).date())

In [16]:
def splitDateAndTime(dateAndTime):
    splitLine = dateAndTime.split(' ')
    return str(splitDate(splitLine[0]))  + splitTime(splitLine[1])

def splitDate(date):
    lineSplit = date.split('/')
    return str(lineSplit[2]) + str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1] if int(lineSplit[1]) >= 10 else '0' + str(lineSplit[1]) )

def splitTime(time):
    lineSplit = time.split(':')
    return str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1])


In [17]:
trip['time'] = trip['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )
trip_test['time'] = trip_test['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )

In [18]:
trip.loc[:,'start_date'] = trip['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))
trip_test.loc[:,'start_date'] = trip_test['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))

In [19]:
trip.loc[:,'day_of_week'] = trip['start_date'].apply(lambda x : x.dayofweek )
trip_test.loc[:,'day_of_week'] = trip_test['start_date'].apply(lambda x : x.dayofweek )

In [20]:
trip.loc[:,'month'] = trip['start_date'].apply(lambda x : x.month )
trip_test.loc[:,'month'] = trip_test['start_date'].apply(lambda x : x.month )

In [21]:
trip.loc[:,'year'] = trip['start_date'].apply(lambda x : x.year )
trip_test.loc[:,'year'] = trip_test['start_date'].apply(lambda x : x.year )

In [22]:
trip.loc[:,'dayofyear'] = trip['start_date'].apply(lambda x : x.dayofyear )
trip_test.loc[:,'dayofyear'] = trip_test['start_date'].apply(lambda x : x.dayofyear )

In [23]:
trip.loc[:,'start_date'] = trip['start_date'].apply(lambda x : pd.to_datetime(x).date())
trip_test.loc[:,'start_date'] = trip_test['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [24]:
cal = calendarUSFH()
holidays = cal.holidays(return_name=True,start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))
holiday_festive_day = pd.DataFrame(holidays,columns=['holiday']).reset_index()
holiday_festive_day.rename(columns={'index':'start_date'},inplace=True)

In [25]:
holiday_festive_day.loc[:,'start_date'] = holiday_festive_day['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [26]:
trip = pd.merge(holiday_festive_day,trip,on=['start_date'],how='right')
trip_test = pd.merge(holiday_festive_day,trip_test,on=['start_date'],how='right')

In [27]:
trip_test.shape

(119998, 11)

In [28]:
trip.loc[:,'is_holiday'] = trip.holiday.isnull()
trip_test.loc[:,'is_holiday'] = trip_test.holiday.isnull()

In [29]:
trip.loc[:,'is_holiday'] = trip['is_holiday'].apply(lambda x : not x )
trip_test.loc[:,'is_holiday'] = trip_test['is_holiday'].apply(lambda x : not x )

In [30]:
trip.loc[:,'subscription_type'] = trip['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )
trip_test.loc[:,'subscription_type'] = trip_test['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )

In [31]:
trip.sample()

,start_date,holiday,id,duration,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
454757,2013-12-02,NaN,112108,279,San Francisco Caltrain (Townsend at 4th),70,0,802,0,12,2013,336,False


In [32]:
trip_test.sample()

,start_date,holiday,id,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
42727,2014-05-29,NaN,302944,Palo Alto Caltrain Station,34,1,1602,3,5,2014,149,False


In [33]:
trip_test.shape

(119998, 12)

In [34]:
tripWithZipCode = pd.merge(trip,stationReduced,on=['station'],how='left')
tripWithZipCode_test = pd.merge(trip_test,stationReduced,on=['station'],how='left')

In [35]:
tripWithZipCode_test.shape

(119998, 13)

In [36]:
tripWithZipCode.sample()

,start_date,holiday,id,duration,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday,zip_code
242355,2015-08-17,NaN,893744,616,Mechanics Plaza (Market at Battery),75,0,1833,0,8,2015,229,False,94107.0


In [37]:
tripWithZipCode_test.sample()

,start_date,holiday,id,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday,zip_code
78155,2014-10-21,NaN,509171,San Francisco Caltrain 2 (330 Townsend),69,0,1851,1,10,2014,294,False,94107


In [38]:
tripWithZipCode = tripWithZipCode.drop(labels=['holiday'],axis=1).dropna()
tripWithZipCode_test = tripWithZipCode_test.drop(labels=['holiday'],axis=1)

In [39]:
tripWithZipCode_test.shape

(119998, 12)

In [40]:
tripWithZipCode.rename(columns={'start_date':'date'},inplace=True)

In [41]:
tripWithZipCode_test.rename(columns={'start_date':'date'},inplace=True)

In [42]:
tripWithZipCode_test.shape

(119998, 12)

In [43]:
tripsWithWeather = pd.merge(tripWithZipCode,weather,on=['date','zip_code'],how='left')
tripsWithWeather_test = pd.merge(tripWithZipCode_test,weather,on=['date','zip_code'],how='left')

In [44]:
reduceWeather = weather.drop(labels=['date','events','zip_code'],axis=1)

In [45]:
for value in reduceWeather.columns :
    tripsWithWeather_test[value] = tripsWithWeather_test[value]\
            .apply(lambda x : tripsWithWeather_test[value].mean() if pd.isnull(x) else x)

In [46]:
tripsWithWeather.rename(columns={'start_station_id':'station_id'},inplace=True)
tripsWithWeather_test.rename(columns={'start_station_id':'station_id'},inplace=True)

In [47]:
tripWithZipCode_test.shape

(119998, 12)

In [48]:
tripsWithWeather = tripsWithWeather.dropna()

In [49]:
tripsWithWeather = tripsWithWeather[tripsWithWeather['duration'] < 5000]

In [50]:
tripsWithWeather.shape

(527164, 35)

In [51]:
tripsWithWeather_test.shape

(119998, 34)

In [52]:
test_id = tripsWithWeather_test['id']

test_target = tripsWithWeather['duration'][-50000:]

In [53]:
train_target = tripsWithWeather['duration']

In [54]:
tripsWithWeatherReduced = tripsWithWeather.drop(labels=['duration','date','station','events','id'],axis=1)

In [55]:
tripsWithWeatherReduced_test = tripsWithWeather_test.drop(labels=['date','station','events','id'],axis=1)

In [56]:
train = tripsWithWeatherReduced
test = tripsWithWeatherReduced_test

# Reduccion de dimensiones 

from sklearn.decomposition import IncrementalPCA

pca = IncrementalPCA(n_components=8)

tripTransform = pca.fit_transform(tripsWithWeatherReduced)
tripTransform_test = pca.fit_transform(tripsWithWeatherReduced_test)

tripTransform.shape

tripTransform_test.shape

tripTransformDF = pd.DataFrame(tripTransform)
tripTransformDF_test = pd.DataFrame(tripTransform_test)

tripTransformDF = tripTransformDF.abs()
tripTransformDF_test = tripTransformDF_test.abs()

train = tripTransformDF

test = tripTransformDF_test

train.shape

train_target.shape

# Machine Learning

In [68]:
from sklearn import ensemble

In [69]:
rfr = ensemble.RandomForestRegressor(n_jobs=-1,n_estimators=50,criterion='mse',oob_score=True)

In [70]:
rfr.fit(train,train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=-1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [71]:
test_prediction = rfr.predict(test)

### Conversion para entrega

In [72]:
predictionDF = pd.DataFrame(test_prediction,columns={"duration"})

In [73]:
predictionDF.loc[:,'id'] = test_id.values

In [74]:
predictionDF.to_csv(path_or_buf='prediction_random_forest',sep=',',header=True,columns=['id','duration'],index=False)

### CrossValidation
#### En caso de querer usar parte del set de entrenamiento como set de prueba 

rfr.score(test,test_target)

from sklearn.metrics import mean_squared_error

mean_squared_error(test_target,test_prediction)

En caso de ser necesario de Cross Validation

predictionDF.loc[:,'spectate'] = test_target.values

def predictError(predict,spectate):
    if (spectate > predict) : return  ((spectate-predict)/spectate)*100
    else  :  return ((predict-spectate)/predict )*100

predictionDF['errorValue'] = predictionDF.apply(axis=1,func=lambda x : predictError(x[0],x[1]))

predictionDF.sample(5)

print 'El error promedio que comete Random Forest Regresion al predecir es de', float(predictionDF['errorValue'].mean()),'%'

print "El algoritmo Random Forest Regresion predice un ",\
float(predictionDF[predictionDF['errorValue'] < 15].shape[0]) / float(predictionDF.shape[0]) *100 ,\
"% de los valores, con un error menor al 15%"


predictionDF.describe()